In [3]:
pip install resnet

  Created wheel for resnet: filename=resnet-0.1-py3-none-any.whl size=10047 sha256=9b01b0b8641d481f0f5125cba3603e395697d10aa3ededce6222d841d6a059d8
  Stored in directory: c:\users\tut\appdata\local\pip\cache\wheels\38\23\58\6e83e34ced7d4772b88e2e86305f94a1ca497bd87937f5e7ec
Successfully built resnet
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install image-classifiers

Note: you may need to restart the kernel to use updated packages.


In [1]:
#from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數

Using TensorFlow backend.


In [2]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [5]:
import tensorflow.keras
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet(input_shape, depth=29, num_classes=10):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [6]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [8]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 47s 947us/sample - loss: 1.8300 - accuracy: 0.5143 - val_loss: 1.8840 - val_accuracy: 0.48486 - loss: 2.4841 - accuracy: 0. - ETA: 1:23 - loss: 2.465 - ETA: 1 - E
Epoch 2/30
50000/50000 [==============================] - 41s 816us/sample - loss: 1.3168 - accuracy: 0.6659 - val_loss: 1.9731 - val_accuracy: 0.4938 accura - ETA: 25s - loss: 1.3947 - accuracy:  - ETA: - ETA: 21 - ETA: 13s - los - ETA: 11s - loss: 1.3452 - accuracy: 0. - ETA: 10s - loss: 1.3447 - accuracy: 0.6 - ETA: 10s - loss: 1.3438 - ETA: 9s - loss: 1.3404 
Epoch 3/30
50000/50000 [==============================] - 41s 817us/sample - loss: 1.1010 - accuracy: 0.7312 - val_loss: 1.7346 - val_accuracy: 0.5771
Epoch 4/30
50000/50000 [==============================] - 41s 817us/sample - loss: 0.9620 - accuracy: 0.7745 - val_loss: 1.3798 - val_accuracy: 0.6383loss: 0.9648 - accu - ETA: 23s - loss: 0.9663 -  - ETA: 13s - 

50000/50000 [==============================] - 41s 818us/sample - loss: 0.4961 - accuracy: 0.9398 - val_loss: 1.2525 - val_accuracy: 0.7552 - loss: 0.4618  - ETA: 28s - loss: 0.4613 - a - ETA: 27s - loss: 0.4625 - accuracy: 0.9 - ETA: 26s - loss: 0.4635 - accu - ETA: 25s - loss: 0.4649 - ac - ETA: 24s - loss: 0.4684  - ETA: 22s - loss: 0.4729 - accuracy: 0.94 - ETA: 22s - l - ETA: 19s - loss: 0.4750 -  - ETA: 5s - loss: 0
Epoch 24/30
50000/50000 [==============================] - 41s 819us/sample - loss: 0.4936 - accuracy: 0.9408 - val_loss: 1.2485 - val_accuracy: 0.7608loss: 0.4456 - accuracy: - ETA: 31s - loss: 0.4465 - accuracy: 0. - ETA: 31s - loss: 0.4449 - accuracy - ETA: 22s - loss: 0.45
Epoch 25/30
50000/50000 [==============================] - 41s 818us/sample - loss: 0.4894 - accuracy: 0.9416 - val_loss: 1.3322 - val_accuracy: 0.7560 loss: 0.4551 - accuracy: 0. - ETA: 31s - loss - ETA: 29s - loss: 0.4550 - accuracy - ETA: 24s - loss: 0.4588 - accurac - ETA: 23s - loss: 0.4616

## 資料增強

使用 ResNet 網路架構 (此處沒有使用 transfer learning，因為沒有使用 pre-trained 的 weights)

學習率動態調整

使用 generator 訓練

本日作業的解答是參考 Keras 官方的範例 - 使用 ResNet 訓練 Cifar-10 資料集，共使用了以下技巧

In [11]:
import tensorflow.compat.v1 as tf
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

config = tf.ConfigProto()

config.gpu_options.per_process_gpu_memory_fraction = 1
sess = tf.Session(config=config)

In [12]:
"""
#Trains a ResNet on the CIFAR10 dataset.

ResNet 共有兩個版本，此處解答我們使用 v1 來做訓練。
ResNet v1:
[Deep Residual Learning for Image Recognition
](https://arxiv.org/pdf/1512.03385.pdf)
ResNet v2:
[Identity Mappings in Deep Residual Networks
](https://arxiv.org/pdf/1603.05027.pdf)
"""

import tensorflow.keras
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
import numpy as np
import os

# 訓練用的超參數
batch_size = 128  
epochs = 200
data_augmentation = True
num_classes = 10

# 資料標準化的方式，此處使用減去所有影像的平均值
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 9 # 使用 ResNet-56 的網路架構

# 使用的 ResNet 模型版本
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# 計算不同 ResNet 版本對應的網路深度，此處都是根據 paper 的定義來計算
depth = n * 6 + 2

# 模型的名稱
model_type = 'ResNet%dv%d' % (depth, version)

# 讀取 Cifar-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 影像輸入的維度
input_shape = x_train.shape[1:]

# 先把影像縮放到 0-1 之間
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# 再減去所有影像的平均值
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean 
    x_test -= x_train_mean # 此處要注意！測試資料也是減去訓練資料的平均值來做標準化，不可以減測試資料的平均值 (因為理論上你是不能知道測試資料的平均值的！)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# 對 label 做 one-hot encoding
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

# 學習率動態調整。當跑到第幾個 epcoh 時，根據設定修改學習率。這邊的數值都是參考原 paper
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

# 使用 resnet_layer 來建立我們的 ResNet 模型
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    # 建立卷積層
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    # 對輸入進行卷機，根據 conv_first 來決定 conv. bn, activation 的順序
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

# Resnet v1 共有三個 stage，每經過一次 stage，影像就會變小一半，但 channels 數量增加一倍。ResNet-20 代表共有 20 層 layers，疊越深參數越多
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]
    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # 模型的初始設置，要用多少 filters，共有幾個 residual block （組成 ResNet 的單元）
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    
    # 建立 Input layer
    inputs = Input(shape=input_shape)
    
    # 先對影像做第一次卷機
    x = resnet_layer(inputs=inputs)
    
    # 總共建立 3 個 stage
    for stack in range(3):
        # 每個 stage 建立數個 residual blocks (數量視你的層數而訂，越多層越多 block)
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y]) # 此處把 featuremaps 與 上一層的輸入加起來 (欲更了解結構需閱讀原論文)
            x = Activation('relu')(x)
        num_filters *= 2

    # 建立分類
    # 使用 average pooling，且 size 跟 featuremaps 的 size 一樣 （相等於做 GlobalAveragePooling）
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    
    # 接上 Dense layer 來做分類
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # 建立模型
    model = Model(inputs=inputs, outputs=outputs)
    return model

# 建立 ResNet v1 模型
model = resnet_v1(input_shape=input_shape, depth=depth)

# 編譯模型，使用 Adam 優化器並使用學習率動態調整的函數，０代表在第一個 epochs
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# 使用動態調整學習率
lr_scheduler = LearningRateScheduler(lr_schedule)

# 使用自動降低學習率 (當 validation loss 連續 5 次沒有下降時，自動降低學習率)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
# 設定 callbacks
callbacks = [lr_reducer, lr_scheduler]


print('Using real-time data augmentation.')
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# 將資料送進 ImageDataGenrator 中做增強
datagen.fit(x_train)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)
Learning rate:  0.001
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_28 (BatchNo (None, 32, 32, 16)   64          conv2d_31[0][0]                  
__________________________________________________________________________________________________
activation_28 (Activation)      (None, 32, 32, 16)   0           batch

In [13]:
# 訓練模型囉！
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)//batch_size),
                    validation_data=(x_test, y_test),
                    epochs=epochs, verbose=1, workers=4,
                    callbacks=callbacks)

# 評估我們的模型
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 390 steps, validate on 10000 samples
Learning rate:  0.001
Epoch 1/200
390/390 [==============================] - 51s 131ms/step - loss: 2.1430 - accuracy: 0.4085 - val_loss: 1.8835 - val_accuracy: 0.4682
Learning rate:  0.001
Epoch 2/200
390/390 [==============================] - 44s 113ms/step - loss: 1.5823 - accuracy: 0.5684 - val_loss: 1.7371 - val_accuracy: 0.5209
Learning rate:  0.001
Epoch 3/200
390/390 [==============================] - 44s 113ms/step - loss: 1.3419 - accuracy: 0.6483 - val_loss: 1.6532 - val_accuracy: 0.5607
Learning rate:  0.001
Epoch 4/200
390/390 [==============================] - 44s 113ms/step - loss: 1.1746 - accuracy: 0.7033 - val_loss: 1.3416 - val_accuracy: 0.6504
Learning rate:  0.001
Epoch 5/200
390/390 [==============================] - 44s 113ms/step - loss: 1.0645 - accuracy: 0.7353 - val_loss: 1.3915 - val_accuracy: 0.6334
Learning rat

390/390 [==============================] - 44s 113ms/step - loss: 0.4351 - accuracy: 0.9182 - val_loss: 0.7753 - val_accuracy: 0.8327
Learning rate:  0.001
Epoch 48/200
390/390 [==============================] - 44s 113ms/step - loss: 0.4404 - accuracy: 0.9173 - val_loss: 0.9430 - val_accuracy: 0.7962
Learning rate:  0.001
Epoch 49/200
390/390 [==============================] - 44s 113ms/step - loss: 0.4329 - accuracy: 0.9193 - val_loss: 0.8261 - val_accuracy: 0.8131
Learning rate:  0.001
Epoch 50/200
390/390 [==============================] - 44s 113ms/step - loss: 0.4329 - accuracy: 0.9199 - val_loss: 1.0017 - val_accuracy: 0.7770
Learning rate:  0.001
Epoch 51/200
390/390 [==============================] - 44s 113ms/step - loss: 0.4294 - accuracy: 0.9203 - val_loss: 0.7129 - val_accuracy: 0.8408
Learning rate:  0.001
Epoch 52/200
390/390 [==============================] - 44s 113ms/step - loss: 0.4274 - accuracy: 0.9216 - val_loss: 0.6911 - val_accuracy: 0.8495
Learning rate:  0.001

390/390 [==============================] - 44s 113ms/step - loss: 0.2161 - accuracy: 0.9862 - val_loss: 0.5116 - val_accuracy: 0.9161
Learning rate:  0.0001
Epoch 96/200
390/390 [==============================] - 44s 113ms/step - loss: 0.2112 - accuracy: 0.9874 - val_loss: 0.5135 - val_accuracy: 0.9130
Learning rate:  0.0001
Epoch 97/200
390/390 [==============================] - 44s 113ms/step - loss: 0.2091 - accuracy: 0.9874 - val_loss: 0.5310 - val_accuracy: 0.9096
Learning rate:  0.0001
Epoch 98/200
390/390 [==============================] - 44s 113ms/step - loss: 0.2077 - accuracy: 0.9871 - val_loss: 0.5077 - val_accuracy: 0.9138
Learning rate:  0.0001
Epoch 99/200
390/390 [==============================] - 44s 113ms/step - loss: 0.2054 - accuracy: 0.9878 - val_loss: 0.5092 - val_accuracy: 0.9171
Learning rate:  0.0001
Epoch 100/200
390/390 [==============================] - 44s 113ms/step - loss: 0.2027 - accuracy: 0.9879 - val_loss: 0.5157 - val_accuracy: 0.9128
Learning rate: 

390/390 [==============================] - 44s 113ms/step - loss: 0.1473 - accuracy: 0.9972 - val_loss: 0.5380 - val_accuracy: 0.9185
Learning rate:  1e-05
Epoch 144/200
390/390 [==============================] - 44s 113ms/step - loss: 0.1479 - accuracy: 0.9966 - val_loss: 0.5371 - val_accuracy: 0.9179
Learning rate:  1e-05
Epoch 145/200
390/390 [==============================] - 44s 113ms/step - loss: 0.1466 - accuracy: 0.9974 - val_loss: 0.5363 - val_accuracy: 0.9181
Learning rate:  1e-05
Epoch 146/200
390/390 [==============================] - 44s 113ms/step - loss: 0.1475 - accuracy: 0.9970 - val_loss: 0.5406 - val_accuracy: 0.9184
Learning rate:  1e-05
Epoch 147/200
390/390 [==============================] - 44s 113ms/step - loss: 0.1472 - accuracy: 0.9967 - val_loss: 0.5397 - val_accuracy: 0.9185
Learning rate:  1e-05
Epoch 148/200
390/390 [==============================] - 44s 113ms/step - loss: 0.1470 - accuracy: 0.9969 - val_loss: 0.5401 - val_accuracy: 0.9190
Learning rate:  

390/390 [==============================] - 44s 113ms/step - loss: 0.1429 - accuracy: 0.9973 - val_loss: 0.5527 - val_accuracy: 0.9186
Learning rate:  5e-07
Epoch 192/200
390/390 [==============================] - 44s 114ms/step - loss: 0.1431 - accuracy: 0.9974 - val_loss: 0.5526 - val_accuracy: 0.9186
Learning rate:  5e-07
Epoch 193/200
390/390 [==============================] - 44s 114ms/step - loss: 0.1426 - accuracy: 0.9976 - val_loss: 0.5529 - val_accuracy: 0.9180
Learning rate:  5e-07
Epoch 194/200
390/390 [==============================] - 44s 113ms/step - loss: 0.1418 - accuracy: 0.9979 - val_loss: 0.5529 - val_accuracy: 0.9184
Learning rate:  5e-07
Epoch 195/200
390/390 [==============================] - 44s 113ms/step - loss: 0.1426 - accuracy: 0.9975 - val_loss: 0.5531 - val_accuracy: 0.9187
Learning rate:  5e-07
Epoch 196/200
390/390 [==============================] - 44s 113ms/step - loss: 0.1426 - accuracy: 0.9975 - val_loss: 0.5528 - val_accuracy: 0.9177
Learning rate:  